# Importing Libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import helper
from helper import *

In [ ]:
import factors
from factors import *

In [ ]:
import importlib
importlib.reload(helper)
importlib.reload(factors)

# Data Collection and Cleaning

In [ ]:
# basic variables

start_date = '2000-01-01'
end_date = '2023-12-31' 
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

In [ ]:
symbols = collect_tickers('US', start_date, end_date)

In [ ]:
snp_symbols = [str(i) for i in symbols]

In [ ]:
snp_stocks_data = download_data(snp_symbols, start_date, end_date)

In [ ]:
snp_stocks_data = clean_data(snp_stocks_data, date_range)
# snp_stocks_data = clean_data(snp_stocks_data, date_range)
# snp_stocks_data = clean_data(snp_stocks_data, date_range)

In [ ]:
symbols = list(snp_stocks_data.columns)

In [ ]:
period = 'W'
time = 'training'

# Calculations

In [ ]:
returns_df = calculate_returns(snp_stocks_data, rebalance=period)

In [ ]:
# if time == 'training':
#     returns_df = returns_df.loc[:, (returns_df.columns > '2005')& (returns_df.columns < '2019')]
# elif time == 'testing':
#     returns_df = returns_df.loc[:,returns_df.columns > '2019']

In [ ]:
'''
A dictionary to store the return%. 
The strategy name is the key and the return-df is value
'''
strategy_returns = {}
strategy_returns_ls = {}

## Momentum Strategy

### Momentum Long Only

In [ ]:
months = [12]
pct = [50]

In [ ]:

# for m in months:
#     momentum = calculate_m_momentum(snp_stocks_data, m)
     
#     for p in pct:
#         strategy = f'{m}_momentum_{p}'        
#         momentum_long(snp_stocks_data, momentum, returns_df, p,  strategy= strategy, strategy_returns=strategy_returns)

In [ ]:
for m in months:
    momentum = calculate_m_momentum(snp_stocks_data, m, resample=period)
    # if time == 'training':
    #     momentum = momentum.loc[:, (momentum.columns > '2005') & (momentum.columns < '2019')]
    # if time == 'testing':
    #     momentum = momentum.loc[:,momentum.columns > '2019']
    for p in pct:
        strategy = f'{m}_momentum_ls_{p}'
        momentum_long_sort(snp_stocks_data, momentum, returns_df, p, p, strategy, strategy_returns=strategy_returns_ls)
        
strategy_returns_ls['12_momentum_ls_50'].rename({0:'12_momentum_ls_50'}, axis=1, inplace=True)

#### 12-1 momentum

In [ ]:
# # Calculate 12-1 momentum for each stock
# momentum_df = calculate_12_1_momentum(snp_stocks_data)
# for p_top in pct:
#     momentum_12_1(snp_stocks_data, momentum_df, returns_df, p_top, strategy_returns=strategy_returns)

In [ ]:
momentum_df = calculate_12_1_momentum(snp_stocks_data, resample=period)
# if time == 'training':
#     returns_df = returns_df.loc[:, (returns_df.columns > '2005')& (returns_df.columns < '2019')]
# elif time == 'testing':
#     momentum_df = momentum_df.loc[:, momentum_df.columns > '2019']

for p_top in pct:
    momentum_12_1_ls(snp_stocks_data, momentum_df, returns_df, p_top, strategy_returns=strategy_returns_ls)

strategy_returns_ls['12_1_momentum_ls_50'].rename({0:'12_1_momentum_ls_50'}, axis=1, inplace=True)

### Reversion Strategy

In [ ]:
# momentum_reversion = calculate_m_momentum(snp_stocks_data, 1)
# for p in pct:
#     strategy = f'{m}-reversal_{p}'
#     reversion_long(snp_stocks_data, momentum_reversion, returns_df, p,  strategy=strategy, strategy_returns=strategy_returns)
        

In [ ]:
momentum_rev = calculate_m_momentum(snp_stocks_data, 1, resample = period)
# if time == 'training':
#     momentum_rev = momentum_rev.loc[:, (momentum_rev.columns>'2005') & (momentum_rev.columns<'2019')]
# elif time == 'testing':
#     momentum_rev = momentum_rev.loc[:, momentum_rev.columns>'2019']

In [ ]:
for p in pct:
        strategy = f'{m}-reversal_ls_{p}'
        reversion_long_sort(snp_stocks_data, momentum_rev, returns_df, p, p, strategy=strategy, strategy_returns=strategy_returns_ls)
            

### Seasonality

1-year check

In [ ]:
# seasonality(snp_stocks_data, returns_df, strategy='1_yr_seasonality', strategy_returns=strategy_returns)

In [ ]:
# seasonality_ls(snp_stocks_data, returns_df, strategy='1_yr_seasonality_ls', strategy_returns=strategy_returns_ls)

3-years seasonality

In [ ]:
# times = [2]
# for m in times:
#     strategy = f'3_yr_seasonality_{m}'
#     seasonality(snp_stocks_data, returns_df, y=3, m = m, n=m, strategy= strategy, strategy_returns=strategy_returns)

In [ ]:
strategy = f'3_yr_seasonality_2_ls'
seasonality_ls(stocks_data=snp_stocks_data, return_df=returns_df, strategy_returns=strategy_returns_ls, rebalance=period, y=3, m = 2, n=2, strategy= strategy, )

5-years seasonality

In [ ]:
# times = [3]
# for m in times:
#     strategy = f'5_yr_seasonality_{m}'
#     seasonality(snp_stocks_data, returns_df, strategy= strategy, strategy_returns=strategy_returns_ls,  y=5, m = m, n=m,)

In [ ]:
times = [3]
for m in times:
    strategy = f'5_yr_seasonality_{m}_ls'
    seasonality_ls(snp_stocks_data, returns_df, strategy_returns=strategy_returns_ls, rebalance=period, y=5, m = m, n=m, strategy= strategy, )

### Value

In [ ]:
# strategy = 'Value < 25'
# price_strategy(snp_stocks_data, returns_df, 25, strategy, strategy_returns=strategy_returns)

In [ ]:
strategy = 'Value_15_85'
price_strategy_ls(snp_stocks_data, returns_df, 85, 15, strategy, strategy_returns=strategy_returns_ls)

In [ ]:
strategy = 'Value_25_75'
price_strategy_ls(snp_stocks_data, returns_df, 75, 25, strategy, strategy_returns=strategy_returns_ls)

### Volatility

In [ ]:
def calculate_vol(return_data, y=1):
    volatility = return_data.T.rolling(window=52*y).std()
    volatility = volatility * np.sqrt(52)
    
    return volatility.T.dropna(axis=1, how='all')

In [ ]:
vol_df = calculate_vol(returns_df)
top = top_p_percentage_stocks(vol_df, 33)
btm = bottom_p_percentage_stocks(vol_df, 33)
ret = portfolio_return(returns_df, top, btm)

In [ ]:
vol_df = calculate_vol(returns_df)
top = top_p_percentage_stocks(vol_df, 50)
btm = bottom_p_percentage_stocks(vol_df, 50)
ret = portfolio_return(returns_df, top, btm)
strategy_returns_ls['1_year_vol_ls_50']  = pd.DataFrame.from_dict(dict(sorted(ret.items())), orient='index', columns=['1_year_vol_ls_50'])

In [ ]:
# for p in pct:
#     # strategy = f'1_year_vol_{p}'
#     volatility_strategy(snp_stocks_data, returns_df, p=p, strategy=strategy, strategy_returns=strategy_returns)

In [ ]:
# for p in pct:
#     strategy = f'1_year_vol_ls_{p}'
#     # volatility_strategy_ls(snp_stocks_data, returns_df, p=p, strategy=strategy, strategy_returns=strategy_returns_ls)

In [ ]:
strategy_returns_ls['1_year_vol_ls_50'][strategy_returns_ls['1_year_vol_ls_50'].index > '2005']

# Metrices

In [ ]:
for key in strategy_returns_ls.keys():
    print(calculate_overall_cagrs(strategy_returns_ls[key], period))
    print(calculate_overall_sharpe_ratios(strategy_returns_ls[key], period))
    print('-----------------------------------------------------------')

In [ ]:
def combine_strategy_returns(strategy_returns_dict):
    # Create an empty list to store DataFrames
    dfs = []
    
    # Iterate over the dictionary
    for strategy, df in strategy_returns_dict.items():
        # Rename the column to the strategy name
        df = df.rename(columns={df.columns[0]: strategy})
        # Append the DataFrame to the list
        dfs.append(df)
    
    # Concatenate all DataFrames in the list along the columns
    combined_df = pd.concat(dfs, axis=1)
    
    return combined_df

In [ ]:
for strategy, df in strategy_returns_ls.items():
    df.columns = [f'{strategy}_{col}' for col in df.columns]

# Combine all DataFrames into a single DataFrame
df = pd.concat(strategy_returns_ls.values(), axis=1)

In [ ]:
df = df[df.index > '2006']

In [ ]:
cagrs = calculate_overall_cagrs(df, 'W')

In [ ]:
sr = calculate_overall_sharpe_ratios(df, 'W')

# Plots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

def calculate_cumulative_returns(df):
    cumulative_returns_df = (1 + df / 100).cumprod() - 1
    return cumulative_returns_df

def plot_strategy_performance(strategy_df):
    # Ensure the index is a datetime index
    strategy_df.index = pd.to_datetime(strategy_df.index)
    
    # Calculate cumulative returns
    cumulative_returns_df = calculate_cumulative_returns(strategy_df)
    
    # Set the plotting style
    sns.set(style="whitegrid")

    # Create individual plots for each strategy
    for strategy in cumulative_returns_df.columns:
        plt.figure(figsize=(14, 7))
        
        # Plot cumulative returns
        plt.plot(cumulative_returns_df.index, cumulative_returns_df[strategy], label=f'{strategy} Cumulative Return')
        
        # Add titles and labels
        plt.title(f'Cumulative Returns of {strategy}')
        plt.xlabel('Date')
        plt.ylabel('Cumulative Return')
        plt.legend(loc='best')
        
        # Show the plot
        plt.show()

plot_strategy_performance(df)


In [ ]:
import plotly.graph_objs as go
import plotly.express as px

def calculate_cumulative_returns(df):
    cumulative_returns_df = (1 + df / 100).cumprod()
    return cumulative_returns_df

def plot_strategy_performance(strategy_df):
    # Ensure the index is a datetime index
    strategy_df.index = pd.to_datetime(strategy_df.index)
    
    # Calculate cumulative returns
    cumulative_returns_df = calculate_cumulative_returns(strategy_df)
    
    # Create the plot
    fig = go.Figure()

    # Add traces for each strategy
    for strategy in cumulative_returns_df.columns:
        fig.add_trace(go.Scatter(
            x=cumulative_returns_df.index,
            y=cumulative_returns_df[strategy],
            mode='lines',
            name=strategy
        ))
    
    # Add titles and labels
    fig.update_layout(
        title='Cumulative Returns of Strategies',
        xaxis_title='Date',
        yaxis_title='Cumulative Return',
        legend_title='Strategies'
    )

    # Show the plot
    fig.show()

plot_strategy_performance(df)
